# 🤗 x 🦾: Training ACT with LeRobot Notebook

Welcome to the **LeRobot ACT training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `ACT` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `ACT` policy for 100,000 steps typically takes **about 1.5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer.

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [2]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 40244, done.
remote: Counting objects: 100% (342/342), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 40244 (delta 253), reused 120 (delta 120), pack-reused 39902 (from 5)
Receiving objects: 100% (40244/40244), 170.00 MiB | 31.10 MiB/s, done.
Resolving deltas: 100% (26260/26260), done.
Filtering content: 100% (45/45), 69.03 MiB | 38.33 MiB/s, done.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ffmpeg=7.1.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attr-2.5.2                 |       h39aace5_0          66 KB  conda-forge

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [1]:
#!wandb login

## Conversion du dataset pour entrainement


In [2]:
# transformation du dataset
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import login

login()

In [4]:
!python -m lerobot.datasets.v30.convert_dataset_v21_to_v30 --repo-id=Heuzef/dataset_exemple_001

Trying to download v3.0 version of the dataset from the hub...
Dataset does not have an uploaded v3.0 version. Continuing with conversion.
Fetching 10 files:   0% 0/10 [00:00<?, ?it/s]
data/chunk-000/episode_000000.parquet:   0% 0.00/14.9k [00:00<?, ?B/s]

videos/chunk-000/observation.images.fron(…):   0% 0.00/3.00M [00:00<?, ?B/s]


.gitattributes: 2.46kB [00:00, 734kB/s]
Fetching 10 files:  10% 1/10 [00:00<00:03,  2.67it/s]


tasks.jsonl: 100% 49.0/49.0 [00:00<00:00, 400kB/s]



info.json: 3.85kB [00:00, 18.9MB/s]



README.md: 4.34kB [00:00, 17.8MB/s]



episodes.jsonl: 100% 70.0/70.0 [00:00<00:00, 661kB/s]



episodes_stats.jsonl: 2.42kB [00:00, 15.2MB/s]



videos/chunk-000/observation.images.top/(…):   0% 0.00/2.70M [00:00<?, ?B/s]



videos/chunk-000/observation.images.outs(…):   0% 0.00/2.96M [00:00<?, ?B/s]

videos/chunk-000/observation.images.fron(…): 100% 3.00M/3.00M [00:00<00:00, 4.00MB/s]



videos/chunk-000/observation.images.top/(…): 100% 2.70M/2.70M [00:00<00:00, 6.20MB

## Gestion dossier outputs


In [5]:
import os
# listing du dossier outputs
!mkdir -p /content/outputs/train
os.chdir("/content/outputs/train/")
!ls

# suppression dossier entrainement si besoin de relancer. Sinon on a une exception
#!rm -r /content/outputs/train/NOM

## Start training ACT with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [6]:
!cd /content/ && python lerobot/src/lerobot/scripts/lerobot_train.py -h

usage: lerobot_train.py [-h] [--config_path str] [--dataset str]
                        [--dataset.repo_id str] [--dataset.root [str]]
                        [--dataset.episodes [List]] [--image_transforms str]
                        [--dataset.image_transforms.enable bool]
                        [--dataset.image_transforms.max_num_transforms int]
                        [--dataset.image_transforms.random_order bool]
                        [--dataset.image_transforms.tfs Dict]
                        [--dataset.revision [str]]
                        [--dataset.use_imagenet_stats bool]
                        [--dataset.video_backend str]
                        [--dataset.streaming bool] [--env str]
                        [--env.type {aloha,pusht,gym_manipulator,libero,metaworld}]
                        [--env.visualization_width int]
                        [--env.visualization_height int] [--robot str]
                        [--env.robot.type {}] [--teleop str]
             

Test if gpu is available

In [7]:
# Vérification de GPU CUDA avec le moins de dépendances possibles

import platform
import sys
import subprocess
import os

print("=== 🔍 INFORMATIONS SYSTÈME ===")
print(f"OS              : {platform.system()} {platform.release()}")
print(f"Version Python  : {sys.version}")
print("-" * 60)

# Bloc 1 : PyTorch (si disponible)
try:
    import torch
    print("=== ⚙️ INFOS CUDA via PyTorch ===")
    print(f"PyTorch version : {torch.__version__}")
    print(f"CUDA disponible : {torch.cuda.is_available()}")
    print(f"CUDA version    : {torch.version.cuda}")
    print(f"CUDNN version   : {torch.backends.cudnn.version() if torch.backends.cudnn.is_available() else 'Non disponible'}")

    if torch.cuda.is_available():
        nb_gpus = torch.cuda.device_count()
        print(f"\nNombre de GPU détectés : {nb_gpus}\n")
        for i in range(nb_gpus):
            props = torch.cuda.get_device_properties(i)
            print(f"--- GPU {i} ---")
            print(f"Nom                : {props.name}")
            print(f"Capacité de calcul : {props.major}.{props.minor}")
            print(f"Mémoire totale     : {props.total_memory / 1024**3:.2f} Go")
            print(f"Processeurs multi  : {props.multi_processor_count}")
            print(f"Fréquence horloge  : {props.clock_rate / 1e3:.0f} MHz")
            print()
    else:
        print("🚫 Aucun GPU CUDA détecté par PyTorch.")

except ImportError:
    print("❌ PyTorch n'est pas installé (import torch impossible).")

print("-" * 60)

# Bloc 2 : TensorFlow (si dispo)
try:
    import tensorflow as tf
    print("=== 🧠 INFOS GPU via TensorFlow ===")
    print(f"TensorFlow version : {tf.__version__}")
    gpus = tf.config.list_physical_devices('GPU')
    print(f"GPU détecté(s) : {len(gpus)}")
    for g in gpus:
        print(f" - {g}")
except ImportError:
    print("❌ TensorFlow n'est pas installé (import tensorflow impossible).")

print("-" * 60)

# Bloc 3 : NVIDIA-SMI (si disponible)
print("=== 🧩 INFOS NVIDIA-SMI ===")
try:
    result = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode == 0:
        print(result.stdout)
    else:
        print("⚠️ La commande 'nvidia-smi' a échoué :")
        print(result.stderr)
except FileNotFoundError:
    print("❌ La commande 'nvidia-smi' n'est pas disponible sur ce système.")

=== 🔍 INFORMATIONS SYSTÈME ===
OS              : Linux 6.6.105+
Version Python  : 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
------------------------------------------------------------
=== ⚙️ INFOS CUDA via PyTorch ===
PyTorch version : 2.8.0+cu126
CUDA disponible : True
CUDA version    : 12.6
CUDNN version   : 91002

Nombre de GPU détectés : 1

--- GPU 0 ---
Nom                : Tesla T4
Capacité de calcul : 7.5
Mémoire totale     : 14.74 Go
Processeurs multi  : 40


AttributeError: 'torch._C._CudaDeviceProperties' object has no attribute 'clock_rate'

In [10]:
!rm -r /content/outputs/train/dataset_exemple_001
!cd /content/ && python lerobot/src/lerobot/scripts/lerobot_train.py \
  --dataset.repo_id=Heuzef/dataset_exemple_001 \
  --policy.type=act \
  --output_dir=outputs/train/dataset_exemple_001  \
  --job_name=dataset_train_001 \
  --policy.device=cuda \
  --wandb.enable=false \
  --policy.repo_id=Heuzef/act_policy_exemple_001 \
  --step=100

INFO 2025-10-18 14:18:09 ot_train.py:163 {'batch_size': 8,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'ColorJitter',
                                                       'weight': 1.0},
                                          'hue': {'kwargs': {'hue': [-0.05,
                 

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli upload ${HF_USER}/il_sim_test0 \
  /content/lerobot/outputs/train/il_sim_test0/checkpoints/last/pretrained_model